In [ ]:
!pip install spacy
!pip install xgboost
!pip install en_core_web_md-3.7.1-py3-none-any.whl
!pip install imblearn
!pip3 install imblearn

In [ ]:
#perform imports and load dataset
import numpy as np
import pandas as pd
import scipy
import spacy
import re # For regular expressions|||
import string # For handling string
import nltk
import pickle
import sklearn
import vaderSentiment
import matplotlib.pyplot as plt
from nltk.util import ngrams
#import seaborn as sns
from nltk.corpus import stopwords
from collections import  Counter
#import gensim
#nltk.download('punkt')
#nltk.download('wordnet')
from matplotlib import pyplot
from numpy import array
#from textblob import TextBlob
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from scipy.sparse import csr_matrix
#from imblearn import under_sampling, over_sampling
#from imblearn.datasets import make_imbalance
#from imblearn.over_sampling import SMOTE
#libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.multiclass import OneVsRestClassifier
 

In [ ]:
#loading the data
data = pd.read_csv("LabData.csv")

In [ ]:
#data preprocessing:
df1 = data
#CLEANING DATA
#drop duplicates
df1 = df1.drop_duplicates(keep='first')
#removing numbers
df1['text'] = df1['text'].astype('string')
df1.dropna(subset=['text'], inplace=True)
#lowercase
df1['text']=df1['text'].apply(lambda x: x.lower())
#removing mentions
df1['text']=df1['text'].apply(lambda x: re.sub("@[A-Za-z0-9_]+",'', x))
#removing numbers and words containing numbers
df1['text']=df1['text'].apply(lambda x: re.sub('\w*\d\w*','', x))
#removing hashtags?
#df1['text']=df1['text'].apply(lambda x: re.sub("#[A-Za-z0-9_]+",'', x))
#df1['text']=df1['text'].apply(lambda x: re.split('#|_', x))
df1['text']=df1['text'].apply(lambda x: " ".join(word.strip() for word in re.split('#|_', x)))
#removing links
df1['text']=df1['text'].apply(lambda x: re.sub(r"http\S+",'', x))
df1['text']=df1['text'].apply(lambda x: re.sub(rwww.\S+,'', x))
#removing punctuation
df1['text']=df1['text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
#removing empty spaces and special characters
df1['text']=df1['text'].apply(lambda x: re.sub('[^A-Za-z0-9]+',' ',x))
#stopwords
nlp = spacy.load('en_core_web_md')
# Print the set of spaCy's default stop words (remember that sets are unordered):
print(nlp.Defaults.stop_words)
#add twitter as a stop word and rt
# Add the word to the set of stop words. Use LOWERCASE!
nlp.Defaults.stop_words.add('twitter')
nlp.Defaults.stop_words.add('rt')
nlp.Defaults.stop_words.add('s')
nlp.Defaults.stop_words.add('tweet')
nlp.Defaults.stop_words.add('trend')
nlp.Defaults.stop_words.add('follower')
nlp.Defaults.stop_words.add('null')
nlp.Defaults.stop_words.add('http')
nlp.Defaults.stop_words.add('url')
nlp.Defaults.stop_words.add('trend')
nlp.Defaults.stop_words.add('trending')
# Set the stop_word tag on the lexeme
nlp.vocab['twitter'].is_stop = True
nlp.vocab['rt'].is_stop = True
nlp.vocab['s'].is_stop = True
nlp.vocab['tweet'].is_stop = True
nlp.vocab['trend'].is_stop = True
nlp.vocab['follower'].is_stop = True
nlp.vocab['null'].is_stop = True
nlp.vocab['http'].is_stop = True
nlp.vocab['url'].is_stop = True
nlp.vocab['trend'].is_stop = True
nlp.vocab['trending'].is_stop = True
stop = nlp.Defaults.stop_words
#lemmatisation
# Loading model
#nlp = spacy.load('en_core_web_sm')

In [ ]:
# Lemmatization with stopwords removal
#df1["text"]=df1["text"].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if ((token.is_stop==False)&((token.pos_=="ADJ")|(token.pos_=="NOUN")|(token.pos_=="VERB")))]))
#df1.head()

In [ ]:
df1["text"]=df1["text"].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if ((token.is_stop==False))]))

In [ ]:
#getting rid of empty text rows
df1 = df1.dropna(subset=['text'])

In [ ]:
data = df1

#### Create DF and TFIDF

In [ ]:
#DF if necessary depending on feature selection
#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfVectorizer
#Create TF vectorizer
#vectorizer = CountVectorizer)
#DF = vectorizer.fit_transform(data['text'])
#DF = pd.DataFrame(DF)
#DF = pd.DataFrame(data=DF, columns=vectorizer.get_feature_names_out())

In [ ]:
#DF.shape

In [ ]:
#del DF

In [ ]:
# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()
# Fit and transform the training data
X_tfidf = vectorizer.fit_transform(data['text'])
X_train_tfidf = X_tfidf.toarray()
#checking type
#test.dtypes()
DF = pd.DataFrame(data=X_train_tfidf, columns=vectorizer.get_feature_names_out())
DF.head
#X_tfidf = X_tfidf.fillna(0)

In [ ]:
# Merge the two DataFrames
DF['text'] = data['text']
DF['label'] = data['label']

In [ ]:
del data

In [ ]:
reducedDF = DF.sample(frac=0.5, random_state=1)

In [ ]:
del DF

In [ ]:
#create train set
#divide into training, validation and test data (DF)
from sklearn.model_selection import train_test_split
ind = reducedDF.index
#sample the indices and then drop from current data set
# Assuming you have features X and labels y
df_train, df_test= train_test_split(ind, test_size=0.5, random_state=42)

In [ ]:
#now create the data sets
TrainSet = reducedDF.drop(df_test)

In [ ]:
del reducedDF

#### Clean the response variable

In [ ]:
#take a look at the Train set
TrainSet.head

In [ ]:
#take a look at the label column and its unique values
TrainSet['label'].unique

In [ ]:
#fix the label column. Ensure labels take on only the values [0,1,-1]
mapping = {'-2': -1, '-1': -1, '0': 0, '1': 1, '9': 0, 'FALSE': 0, 'TRUE': 0, 'lab': 0}
TrainSet['label'] = TrainSet['label'].map(mapping)
#TrainSet.fillna(0)

In [ ]:
#check that the mapping worked
TrainSet['label'].unique

In [ ]:
#TrainSet.dropna(subset= "label")

In [ ]:
del TrainSet['text']

In [ ]:
TrainSet.shape

#### Split the data

In [ ]:
#split into train and test set
#training set
y_train = TrainSet["label"]

In [ ]:
y_train = y_train.fillna(0)

In [ ]:
y_train.unique()

In [ ]:
del TrainSet['label']

In [ ]:
#ValSet = pd.read_csv("ReducedData.csv")
ValSet = pd.read_csv("TestReducedtfidf.csv")

#### Do the same for Val set

In [ ]:
#Letting the label column be values [0,-1,1]
mapping = {'-2': -1, '-1': -1, '0': 0, '1': 1, '9': 0, 'FALSE': 0, 'TRUE': 0, 'lab': 0}
ValSet['label'] = ValSet['label'].map(mapping)
ValSet['label']  = ValSet['label'].fillna(0)

In [ ]:
ValSet["label"].unique()

In [ ]:
y_val = ValSet["label"]
y_val.unique()

In [ ]:
#delete label and text columns 
del ValSet['Unnamed: 0']
del ValSet['Unnamed: 0.1']
del ValSet['text']
del ValSet['label']

In [ ]:
ValSet.shape

In [ ]:
TrainSet.shape

#### Random forest hyperparameter tuning

In [ ]:
# libraries
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
import pandas as pd
from sklearn.metrics import confusion_matrix
 
# grid search
#nfeatures = [1000]
#nfeatures = [2000]
nfeatures = [1000] #4000, 5000, 6000, 8000, 10000]
rfn = [50]
max_values = [10]
 
# setting the sets
#Xtrain = TrainSet
#XVal = ValSet
#ytrain = yX[0:1000]
#ytrain = y_train
#yval = yV[0:1000]
#yval = y_val
 
# setting objects to store performance results
results = []
 
for nf in nfeatures:
    for n in rfn:
        for m in max_values:
            # Random forest classifier
            # Feature selection
            chi2_features = SelectKBest(score_func=chi2, k=nf)
            ChiX_selected = chi2_features.fit_transform(TrainSet, y_train)
            support = chi2_features.get_support()
 
            # Get the selected feature names using the support indices
            selected_feature_names = TrainSet.columns[support]
            X_selected = TrainSet[selected_feature_names]
 
            # Random Oversampling
            ros = RandomOverSampler(random_state=0)
            X_oversampled1, y_oversampled1 = ros.fit_resample(X_selected, y_train)
 
            # Fit the Random Forest model to the training data
            RFC1 = RandomForestClassifier(n_estimators=n, max_depth=m)
            RFC1.fit(X_oversampled1, y_oversampled1)
 
            # equate the column variables for train and validation (since we've done a FS)
            test = pd.DataFrame(TestSet)
            test_cols = TestSet.columns
            common_cols = selected_feature_names.intersection(test_cols)
            testnew = TestSet[common_cols]
 
            # Predict the labels of the testing data
            y_pred1 = RFC1.predict(testnew)
 
            # Print the accuracy of the model
            accuracy = accuracy_score(y_val, y_pred1)
 
            # Print the MSE of the model
            conf_matrix =  confusion_matrix(y_val, y_pred1)
            mcr = (conf_matrix[0, 1] + conf_matrix[1, 0]) / len(y_val)
 
            # keep record of the parameters and results
            results.append({
                'nfeatures': nf,
                'n_estimators': n,
                'ax_depth': m,
                'accuracy': accuracy,
                'mcr': mcr
            })

# print the results
for result in results:
    print(result)
 
 

In [ ]:
# print the results
results = pd.DataFrame(results)
results.to_excel("RFResults2.xlsx")

In [ ]:
#import test set
import pandas as pd
TestSet = pd.read_csv("TestReducedtfidf.csv")
TestSet.shape
 

In [ ]:
mapping = {'-2': -1, '-1': -1, '0': 0, '1': 1, '9': 0, 'FALSE': 0, 'TRUE': 0, 'lab': 0}
TestSet['label'] = TestSet['label'].map(mapping)
TestSet['label']  = TestSet['label'].fillna(0)

In [ ]:
TestSet["label"].unique()

In [ ]:
y_test = TestSet["label"]

In [ ]:
#del TestSet['Unnamed: 0']
#del TestSet['Unnamed: 0.1']
del TestSet['text']
del TestSet['label']

In [ ]:
# libraries
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
import pandas as pd
from sklearn.metrics import confusion_matrix
 
# Random forest classifier
# Feature selection
chi2_features = SelectKBest(score_func=chi2, k=1000)
ChiX_selected = chi2_features.fit_transform(TrainSet, y_train)
support = chi2_features.get_support()
 
# Get the selected feature names using the support indices
selected_feature_names = TrainSet.columns[support]
X_selected = TrainSet[selected_feature_names]
 
# Random Oversampling
ros = RandomOverSampler(random_state=0)
X_oversampled1, y_oversampled1 = ros.fit_resample(X_selected, y_train)
 
# Fit the Random Forest model to the training data
RFC1 = RandomForestClassifier(n_estimators=50, max_depth=10)
RFC1.fit(X_oversampled1, y_oversampled1)
 
# equate the column variables for train and validation (since we've done a FS)
test = pd.DataFrame(TestSet)
test_cols = TestSet.columns
common_cols = selected_feature_names.intersection(test_cols)
testnew = TestSet[common_cols]
 
# Predict the labels of the testing data
y_pred1 = RFC1.predict(testnew)
 
# Print the accuracy of the model
accuracy = accuracy_score(y_test, y_pred1)
 
# Calculate MCR for three-class problem
conf_matrix =  confusion_matrix(y_test, y_pred1)
mcr = (conf_matrix[0, 1] + conf_matrix[0, 2] + conf_matrix[1, 0] + conf_matrix[1, 2] + conf_matrix[2, 0] + conf_matrix[2, 1]) / len(y_test)
 
from sklearn.metrics import f1_score
f1 = f1_score(y_test, y_pred1, average="weighted")

In [ ]:
mcr

In [ ]:
conf_matrix

In [ ]:
accuracy

In [ ]:
f1

#### SVM hyperparameter tuning

In [ ]:
# libraries
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
import pandas as pd
from sklearn.metrics import confusion_matrix
 
# grid search
nfeatures = [2000]#, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]
kernel = ['poly', 'rbf', 'sigmoid']
C = [50, 10]#, 1.0] #, 0.1, 0.01]
 
# setting the sets
#Xtrain = TrainSet[0:100]
#XVal = ValSet[0:100]
#ytrain = yX[0:1000]
#ytrain = y_train[0:100]
#yval = yV[0:1000]
#yval = y_val[0:100]
 
# setting objects to store performance results
results2 = []
 
for nf in nfeatures:
    for k in kernel:
        for c in C:
            # Random forest classifier
            # Feature selection
            svd = TruncatedSVD(n_components=nf)
            X_svd = svd.fit_transform(TrainSet)
 
            # Random Oversampling
            ros = RandomOverSampler(random_state=0)
            X_oversampled1, y_oversampled1 = ros.fit_resample(X_svd, y_train)
 
            # Fit the Random Forest model to the training data
            SVC1 = OneVsRestClassifier(SVC(kernel = k, C=c, gamma="auto"))
            SVC1.fit(X_oversampled1, y_oversampled1)
 
            # Predict the labels of the testing data
           
            testnew = svd.transform(ValSet)
            y_pred1 = SVC1.predict(testnew)
 
            # Print the accuracy of the model
            accuracy = accuracy_score(y_val, y_pred1)
 
            # Print the MSE of the model
            conf_matrix =  confusion_matrix(y_val, y_pred1)
            mcr = (conf_matrix[0, 1] + conf_matrix[0, 2] + conf_matrix[1, 0] + conf_matrix[1, 2] + conf_matrix[2, 0] + conf_matrix[2, 1]) / len(y_val)
 
            # keep record of the parameters and results
            results2.append({
                'nfeatures': nf,
                'kernel':k,
                'C': c,
                'accuracy': accuracy,
                'mcr': mcr
            })
 
# print the results
for result in results2:
    print(result)

In [ ]:
# print the results
results = pd.DataFrame(results2)
results.to_excel("SVCResults.xlsx")

In [ ]:
# libraries
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
import pandas as pd
from sklearn.metrics import confusion_matrix
 
# grid search
nfeatures = [1000]#, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]
kernel = ['poly', 'rbf', 'sigmoid']
#C = [50, 10, 1.0,
C= [0.1, 0.01]
 
# setting the sets
#Xtrain = TrainSet[0:100]
#XVal = ValSet[0:100]
#ytrain = yX[0:1000]
#ytrain = y_train[0:100]
#yval = yV[0:1000]
#yval = y_val[0:100]
 
# setting objects to store performance results
results2 = []
 
for nf in nfeatures:
    for k in kernel:
        for c in C:
            # Random forest classifier
            # Feature selection
            svd = TruncatedSVD(n_components=nf)
            X_svd = svd.fit_transform(TrainSet)
 
            # Random Oversampling
            ros = RandomOverSampler(random_state=0)
            X_oversampled1, y_oversampled1 = ros.fit_resample(X_svd, y_train)
 
            # Fit the Random Forest model to the training data
            SVC1 = OneVsRestClassifier(SVC(kernel = k, C=c, gamma="auto"))
            SVC1.fit(X_oversampled1, y_oversampled1)
 
            # Predict the labels of the testing data
            testnew = svd.transform(ValSet)
            y_pred1 = SVC1.predict(testnew)
 
            # Print the accuracy of the model
            accuracy = accuracy_score(y_val, y_pred1)
 
            # Print the MSE of the model
            conf_matrix =  confusion_matrix(y_val, y_pred1)
            mcr = (conf_matrix[0, 1] + conf_matrix[1, 0]) / len(y_val)
 
            # keep record of the parameters and results
            results2.append({
                'nfeatures': nf,
                'kernel':k,
                'C': c,
                'accuracy': accuracy,
                'mcr': mcr
            })
 
# print the results
for result in results2:
    print(result)
 


In [ ]:
# print the results
results = pd.DataFrame(results2)
results.to_excel("SVCResults3.xlsx")
 

#### XGBoost hyperparameter tuning

In [ ]:
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_val_enc = le.transform(y_val)

In [ ]:
#label = 0:negative, 1:neutral, 2:positive
# libraries
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
import pandas as pd
 
# grid search
nfeatures = [5000] #, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]
maxdepth = [None,5, 10]
n_estimators= [200]#50]#,100,200]
learning_rate = [0.01]#0.001]#, 0.01, 0.1]
subsample = [0.5, 0.7, 1.0]
 
# setting objects to store performance results
results3 = []
 
for nf in nfeatures:
    for m in maxdepth:
        for l in learning_rate:
            for ss in subsample:
                for n in n_estimators:
                    # XGboost
                     # Feature selection
                    svd = TruncatedSVD(n_components=nf)
                    X_svd = svd.fit_transform(TrainSet)
 
                    # Random Oversampling
                    ros = RandomOverSampler(random_state=0)
                    X_oversampled1, y_oversampled1 = ros.fit_resample(X_svd, y_train_enc)
                    del X_svd
 
                    # Fit the Random Forest model to the training data
                    XGB1 = XGBClassifier(max_depth = m, learning_rate=l, n_estimators = n, subsample=ss)
                    XGB1.fit(X_oversampled1, y_oversampled1)
                    del X_oversampled1
                    del y_oversampled1
 
                    # Predict the labels of the testing data
                    testnew = svd.transform(ValSet)
                    y_pred1 = XGB1.predict(testnew)
 
                    # Print the accuracy of the model
                    accuracy = accuracy_score(y_val_enc, y_pred1)
 
                    #maybe add a more sensitive performance metric
 
                    # Print the MSE of the model
                    conf_matrix =  confusion_matrix(y_val_enc, y_pred1)
                    mcr = (conf_matrix[0, 1] + conf_matrix[0, 2] + conf_matrix[1, 0] + conf_matrix[1, 2] + conf_matrix[2, 0] + conf_matrix[2, 1]) / len(y_val_enc)
 
                    del XGB1
                    del y_pred1
                    del conf_matrix
                    print("step 4 done")
                    # keep record of the parameters and results
                    results3.append({
                        'nfeatures': nf,
                        'learning_rate': l,
                        'max_depth': m,
                        'subsample': ss,
                        'accuracy': accuracy,
                        'mcr': mcr
                    })
 
# print the results
for result in results3:
    print(result)

In [ ]:
# print the results
results = pd.DataFrame(results3)
results.to_excel("XGResults3.xlsx")

#### Run selected models on test set

##### Load test set

In [ ]:
#import test set
import pandas as pd
TestSet = pd.read_csv("TestReducedtfidf.csv")
TestSet.shape

In [ ]:
#clean respone variable
mapping = {'-2': -1, '-1': -1, '0': 0, '1': 1, '9': 0, 'FALSE': 0, 'TRUE': 0, 'lab': 0}
TestSet['label'] = TestSet['label'].map(mapping)
TestSet['label']  = TestSet['label'].fillna(0)

In [ ]:
TestSet["label"].unique()

In [ ]:
y_test = TestSet["label"]

In [ ]:
y_test.unique()

In [ ]:
#del TestSet['Unnamed: 0']
#del TestSet['Unnamed: 0.1']
del TestSet['text']

In [ ]:
del TestSet['label']

In [ ]:
TestSet.shape

In [ ]:
#TestSet = TestSet.fillna(0)
TestSet

##### Test on Random Forest on test set

In [ ]:
# libraries
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
import pandas as pd
from sklearn.metrics import confusion_matrix
 
# Random forest classifier
# Feature selection
chi2_features = SelectKBest(score_func=chi2, k=1000)
ChiX_selected = chi2_features.fit_transform(TrainSet, y_train)
support = chi2_features.get_support()
 
# Get the selected feature names using the support indices
selected_feature_names = TrainSet.columns[support]
X_selected = TrainSet[selected_feature_names]
 
# Random Oversampling
ros = RandomOverSampler(random_state=0)
X_oversampled1, y_oversampled1 = ros.fit_resample(X_selected, y_train)
 
# Fit the Random Forest model to the training data
RFC1 = RandomForestClassifier(n_estimators=50, max_depth=10)
RFC1.fit(X_oversampled1, y_oversampled1)
 
# equate the column variables for train and validation (since we've done a FS)
test = pd.DataFrame(TestSet)
test_cols = TestSet.columns
common_cols = selected_feature_names.intersection(test_cols)
testnew = TestSet[common_cols]
 
# Predict the labels of the testing data
y_pred1 = RFC1.predict(testnew)
 
# Print the accuracy of the model
accuracy = accuracy_score(y_test, y_pred1)
 
# Calculate MCR for three-class problem
conf_matrix =  confusion_matrix(y_test, y_pred1)
mcr = (conf_matrix[0, 1] + conf_matrix[0, 2] + conf_matrix[1, 0] + conf_matrix[1, 2] + conf_matrix[2, 0] + conf_matrix[2, 1]) / len(y_test)
 
from sklearn.metrics import f1_score
f1 = f1_score(y_test, y_pred1, average="weighted")

In [ ]:
mcr

In [ ]:
conf_matrix

In [ ]:
accuracy

In [ ]:
f1

##### Test on SVM on test set

In [ ]:
#Run SVC on test set
 
# libraries
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
import pandas as pd
from sklearn.metrics import confusion_matrix
 
# Random forest classifier
# Feature selection
svd = TruncatedSVD(n_components=750)
X_svd = svd.fit_transform(TrainSet)
 
# Random Oversampling
ros = RandomOverSampler(random_state=0)
X_oversampled1, y_oversampled1 = ros.fit_resample(X_svd, y_train)
 
# Fit the Random Forest model to the training data
SVC1 = OneVsRestClassifier(SVC(kernel = "poly", C=0.1, gamma="auto"))
SVC1.fit(X_oversampled1, y_oversampled1)
 
# Predict the labels of the testing data
testnew = svd.transform(TestSet)
y_pred1 = SVC1.predict(testnew)
 
# Print the accuracy of the model
accuracy = accuracy_score(y_test, y_pred1)
 
# Print the MSE of the model
conf_matrix =  confusion_matrix(y_test, y_pred1)
 
# Calculate MCR for three-class problem
mcr = (conf_matrix[0, 1] + conf_matrix[0, 2] + conf_matrix[1, 0] + conf_matrix[1, 2] + conf_matrix[2, 0] + conf_matrix[2, 1]) / len(y_test)
 
from sklearn.metrics import f1_score
f1 = f1_score(y_test, y_pred1, average="weighted")

In [ ]:
f1

In [ ]:
accuracy

In [ ]:
mcr

In [ ]:
conf_matrix

#### Testing XGBoost on test data

In [ ]:
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_val_enc = le.transform(y_val)

In [ ]:
ValSet

In [ ]:
ValSet = ValSet.drop(columns='text')

In [ ]:
#label = 0:negative, 1:neutral, 2:positive
# libraries
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
import pandas as pd
 
nf = 500
# Feature selection
svd = TruncatedSVD(n_components=nf)
X_svd = svd.fit_transform(TrainSet)
 
# Random Oversampling
ros = RandomOverSampler(random_state=0)
X_oversampled1, y_oversampled1 = ros.fit_resample(X_svd, y_train_enc)
del X_svd
 
# Fit the Random Forest model to the training data
XGB1 = XGBClassifier(max_depth = 10, learning_rate=0.1, n_estimators = 200, subsample=0.5)
XGB1.fit(X_oversampled1, y_oversampled1)
del X_oversampled1
del y_oversampled1
 
# Predict the labels of the testing data
testnew = svd.transform(ValSet)
y_pred1 = XGB1.predict(testnew)
 
# Print the accuracy of the model
accuracy = accuracy_score(y_val_enc, y_pred1)
 
#maybe add a more sensitive performance metric
 
# Print the MSE of the model
conf_matrix =  confusion_matrix(y_val_enc, y_pred1)
mcr = (conf_matrix[0, 1] + conf_matrix[0, 2] + conf_matrix[1, 0] + conf_matrix[1, 2] + conf_matrix[2, 0] + conf_matrix[2, 1]) / len(y_val_enc)
 
 
from sklearn.metrics import f1_score
f1 = f1_score(y_val_enc, y_pred1, average="weighted")
 

In [ ]:
mcr

In [ ]:
accuracy

In [ ]:
f1